*0. Preamble*

In [1]:
%run stdPackages.ipynb
slides = True # print for slides

# Plots and simulations for L2

## 1. Fuel cost projections and emissions (KF21)

Load data:

In [2]:
db = pyDbs.read.dbFromWB(os.path.join(d['data'],'FuelProjections.xlsx'), {'variable2D': ['prices','emissionIntensity']})
pyDbs.readSets(db) # read set definitions from other symbols declared in the database

Group variables for the plots:

In [3]:
groups = {'Fossil': pd.Index(['Coal','Oil','Natural gas'],name='BFt'),
          'Biomass': pd.Index(['Straw','Wood pellets','Wood chips','Wood waste'], name = 'BFt'),
          'Other': pd.Index(['Waste','Uranium','Hydrogen','Biogas'], name = 'BFt')
    }

Small adjustment to data:

In [4]:
db['FuelPrice'].index = db['FuelPrice'].index.set_levels(db['t'].astype(int), level='t')
db['FuelPrice'] = db['FuelPrice'].unstack('t').sort_values(by=2020, ascending=False).stack()

Plot:

In [5]:
%%capture
mult_graphs()
fig, axes = plt.subplots(1, 3, figsize = (14,8));
plt.subplots_adjust(hspace=0.35)
for j in range(3):
    ax = plt.subplot(1, 3, j+1)
    g, ind = list(groups.keys())[j], list(groups.values())[j]
    seaborn.lineplot(data=pyDbs.adj.rc_pd(db['FuelPrice'], ind).unstack('BFt').rename_axis(columns=None), ax = ax, linewidth = 2);
    ax.set_ylim([0,30]);
    ax.set_ylabel('€/GJ');
    ax.set_xlabel('');
    ax.set_title(g)
fig.tight_layout()
if slides:
    fig.savefig(os.path.join(d['figs'],"L3_FuelCosts_slides.pdf"),facecolor='#FAFAFA',edgecolor='k')
else:
    fig.savefig(os.path.join(d['figs'],"L3_FuelCosts.pdf"),edgecolor='k')

Emission intensity:

In [6]:
%%capture 
mult_graphs()
nplots = 2
nrows = math.ceil(nplots/2)
fig, axes = plt.subplots(nrows, min(nplots, 2), figsize = (14, (6*nrows)));
ax = plt.subplot(nrows,2,1)
e = 'CO2'
db['EmissionIntensity'].xs(e)[db['EmissionIntensity'].xs(e)!=0].sort_values().plot.bar(ax=ax);
ax.set_ylabel('kg/GJ');
ax.set_title(e);
ax = plt.subplot(nrows,2,2)
e = 'SO2'
db['EmissionIntensity'].xs(e)[db['EmissionIntensity'].xs(e)!=0].sort_values().plot.bar(ax=ax);
ax.set_ylabel('kg/GJ');
ax.set_title(e);
fig.tight_layout();
if slides:
    fig.savefig(os.path.join(d['figs'],"L3_emissionIntensity_slides.pdf"),facecolor='#FAFAFA',edgecolor='k')
else:
    fig.savefig(os.path.join(d['figs'],"L3_emissionIntensity.pdf"),edgecolor='k')

## 2. mBasic

Load data:

In [7]:
kwargs = {'variables':  ['Fundamentals', 'Load','Generators_Other'],
          'variable2D': ['Generators_FuelMix'],
          'scalars': ['Scalars'],
          'maps': ['Generators_Categories']}
db = pyDbs.read.dbFromWB(os.path.join(d['data'], 'mBasic.xlsx'), kwargs)
pyDbs.readSets(db)

Set up model and solve:

In [8]:
m = mBasic.mSimple(db)
m() # go through standard steps

Solution status 0: Optimization terminated successfully. (HiGHS Status 7: Optimal)


Drawing the merit-order curve (by hand) is very straightforward, but plotting it here is actually quite a cumbersome exercise. This shows one implementation:

In [9]:
df = pd.concat([db['mc'].rename('mc'), db['GeneratingCapacity'].rename('q')], axis =1).sort_values(by='mc')
df['q'] = df['q'].cumsum()
df.loc['_0'] = [df['mc'].iloc[0], 0] 
df.loc['_E'] = [df['mc'].max()+4, df['q'][-2]+np.finfo(float).eps]
df = df.sort_values(by=['mc','q'])
df['aux'] = df.apply(lambda x, shift: np.roll(x, shift)+np.finfo(float).eps, shift=1)['q']
df = pd.concat([df[['mc','q']].iloc[0:-1], df[['mc','aux']].iloc[1:].rename(columns={'aux':'q'})]).sort_values(by=['mc','q']).set_index('q')
df.loc[0] = 0 # 

This draws up a somewhat fancy plot with annotations and such

In [10]:
%%capture
one_graph()
if slides:
    fig, ax = plt.subplots(1,1,figsize = (14,10))
else:
    fig, ax = plt.subplots(1,1,figsize = (14,8))    
df.plot(linewidth=3, ax=ax, legend=False);
ax.vlines(db['Load'].sum(), 0, math.ceil(df.max())+4,linewidth=3, color = colors[1]);
ax.set_xlabel(r'$GJ$', labelpad = 5);
ax.set_ylabel(r'$€/$GJ', labelpad = 5);
ax.set_xlim([0, df.index.max()-1]);
ax.set_ylim([0, math.ceil(df.max())]);
plt.text(df.index[2]/2-3, df.iloc[2]+0.25, f'Wind');
plt.text(df.index[2]/2-2, df.iloc[2]-1, f'$q_W$');
plt.annotate("",xy=(1, df.iloc[2]-0.5), xytext=(df.index[2]-1, df.iloc[2]-0.5), arrowprops=dict(width=1, color='k', headwidth=5, headlength=15));
plt.annotate("",xy=(df.index[2]-1, df.iloc[2]-0.5), xytext=(1, df.iloc[2]-0.5), arrowprops=dict(width=1, color='k', headwidth=5, headlength=15));

plt.text((df.index[4]+df.index[2])/2-3, df.iloc[4]+0.25, f'Coal')
plt.text((df.index[4]+df.index[2])/2-2, df.iloc[4]-1, f'$q_C$');
plt.annotate("",xy=(df.index[2]+1, df.iloc[4]-0.5), xytext=(df.index[4]-1,df.iloc[4]-0.5), arrowprops=dict(width=1, color='k', headwidth=5, headlength=15));
plt.annotate("",xy=(df.index[4]-1, df.iloc[4]-0.5), xytext=(df.index[2]+1, df.iloc[4]-0.5), arrowprops=dict(width=1, color='k', headwidth=5, headlength=15));

plt.text((df.index[6]+df.index[4])/2-3, df.iloc[6]+0.25, f'Gas')
plt.text((df.index[6]+df.index[4])/2-2, df.iloc[6]-1, f'$q_G$');
plt.annotate("",xy=(df.index[4]+1, df.iloc[6]-0.5), xytext=(df.index[6]-1,df.iloc[6]-0.5), arrowprops=dict(width=1, color='k', headwidth=5, headlength=15));
plt.annotate("",xy=(df.index[6]-1, df.iloc[6]-0.5), xytext=(df.index[4]+1, df.iloc[6]-0.5), arrowprops=dict(width=1, color='k', headwidth=5, headlength=15));

plt.text((df.index[8]+df.index[6])/2-3, df.iloc[8]+0.25, f'Bio')
plt.text((df.index[8]+df.index[6])/2-2, df.iloc[8]-1, f'$q_B$');
plt.annotate("",xy=(df.index[6]+1, df.iloc[8]-0.5), xytext=(df.index[8]-1,df.iloc[8]-0.5), arrowprops=dict(width=1, color='k', headwidth=5, headlength=15));
plt.annotate("",xy=(df.index[8]-1, df.iloc[8]-0.5), xytext=(df.index[6]+1, df.iloc[8]-0.5), arrowprops=dict(width=1, color='k', headwidth=5, headlength=15));


fig.legend(['Generation', 'Load'],loc=9,ncol=2,frameon=True)
fig.tight_layout();
fig.subplots_adjust(top=0.9);
if slides:
    fig.savefig(os.path.join(d['figs'],"L3_MeritOrder_slides.pdf"),facecolor='#FAFAFA',edgecolor='k')
else:
    fig.savefig(os.path.join(d['figs'],"L3_MeritOrder.pdf"),edgecolor='k')

### 2.1. Cap emissions

Define emissions cap model and solve:

In [11]:
m = mBasic.mEmissionCap(db)
m()

Solution status 0: Optimization terminated successfully. (HiGHS Status 7: Optimal)


In [12]:
baselineEmissions = m.db['Emissions'][0]

System costs as a function of cap:

In [13]:
loop = pd.Index(range(51), name = 'loop')
v0, vT = baselineEmissions, 0
capGrid = pyDbs.adjMultiIndex.addGrid(v0,vT,loop,'CO2Cap')
extract = ['FuelConsumption','Emissions','Welfare','Generation']

We run the experiment using the ```loopSolvExtract``` method:

In [14]:
solution = m.loopSolveExtract(loop,[capGrid],extract)

Add wind to fuel consumption:

In [15]:
solution['FuelConsumption'] = solution['FuelConsumption'].unstack('BFt').assign(Wind = solution['Generation'].xs('Wind turbine')).stack().reorder_levels(solution['FuelConsumption'].index.names)

*Plot system costs next to the fuel consumption split:*

In [16]:
SC = pd.Series(solution['Welfare'].values, index = capGrid.values)
FC = pd.Series(solution['FuelConsumption'].values, index = solution['FuelConsumption'].index.set_levels(capGrid.values, level = 1))
FC_df = FC.unstack('BFt').reindex(columns = ['Wind']+[k for k in FC.index.levels[0] if k != 'Wind'])

*PLot with fuel consumption:*

In [17]:
%%capture
mult_graphs()
nplots = 2
nrows = math.ceil(nplots/2)
if slides:
    fig, axes = plt.subplots(nrows, min(nplots, 2), figsize = (14, 8))
else:
    fig, axes = plt.subplots(nrows, min(nplots, 2), figsize = (14, (6*nrows)));
plt.subplots_adjust(hspace=0.35)
# Plot 1:
ax = plt.subplot(nrows, min(nplots,2), 1)
seaborn.lineplot(data = SC, linewidth = 3, ax = ax, legend=False);
ax.set_xlim([0, baselineEmissions]);
ax.set_xlabel(r'$\overline{CO2}$', labelpad=10);
ax.set_ylabel(r"Welfare", labelpad=10);

# Plot 2: Zooming in
ax = plt.subplot(nrows, min(nplots,2), 2)
FC_df.plot.area(ax = ax, legend=False, linewidth=0);
ax.set_xlim([0,baselineEmissions]);
ax.set_xlabel(r'$\overline{CO2}$', labelpad=10);
ax.set_ylabel(r"GJ fuel",labelpad=10);
handles, labels = ax.get_legend_handles_labels()
leg = ax.legend(handles[::-1], labels[::-1], loc=7, bbox_to_anchor = (1.4,0.5))
fig.tight_layout()
if slides:
    fig.savefig(os.path.join(d['figs'], "L3_mBasicCap_slides.pdf"), facecolor='#FAFAFA',edgecolor='k')
else:
    fig.savefig(os.path.join(d['figs'], "L3_mBasicCap.pdf"),edgecolor='k')

### 2.2. Compare with RES target

Add variable 'RESCap' to database:

In [18]:
db_RES = db.copy()
db_RES['RESCap'] = 0

Set up and solve model:

In [19]:
m_RES = mBasic.mRES(db_RES)
m_RES()

Solution status 0: Optimization terminated successfully. (HiGHS Status 7: Optimal)


System costs as a function of RESCap:

In [20]:
loop = pd.Index(range(51), name = 'loop')
v0, vT = 0, 1
capGrid = pyDbs.adjMultiIndex.addGrid(v0,vT,loop,'RESCap')
extract = ['FuelConsumption','Emissions','Welfare','Generation']

We run the experiment using the ```loopSolvExtract``` method:

In [21]:
solution_RES = m_RES.loopSolveExtract(loop,[capGrid],extract)

Add wind to fuel consumption:

In [22]:
solution_RES['FuelConsumption'] = solution_RES['FuelConsumption'].unstack('BFt').assign(Wind = solution_RES['Generation'].xs('Wind turbine')).stack().reorder_levels(solution_RES['FuelConsumption'].index.names)

Swap the grid of RESCap out with resulting emissions:

In [23]:
capGrid = solution_RES['Emissions'].xs('CO2')

*Plot system costs next to the fuel consumption split; add system costs of emission cap scenario as well:*

In [24]:
SC_RES = pd.Series(solution_RES['Welfare'].values, index = capGrid.values)
FC_RES = pd.Series(solution_RES['FuelConsumption'].values, index = solution_RES['FuelConsumption'].index.set_levels(capGrid.values, level = 1,verify_integrity = False))
FC_RES_df = FC_RES.unstack('BFt').reindex(columns = ['Wind']+[k for k in FC_RES.index.levels[0] if k != 'Wind'])

*PLot with fuel consumption:*

In [25]:
%%capture
mult_graphs()
nplots = 2
nrows = math.ceil(nplots/2)
if slides:
    fig, axes = plt.subplots(nrows, min(nplots, 2), figsize = (14, 8));
else:
    fig, axes = plt.subplots(nrows, min(nplots, 2), figsize = (14, (6*nrows)));
plt.subplots_adjust(hspace=0.35)
# Plot 1:
ax = plt.subplot(nrows, min(nplots,2), 1)
SC.plot(ax=ax, linewidth=3, legend =False);
SC_RES.plot(ax=ax,linewidth=3,legend=False);
ax.set_xlim([0, baselineEmissions]);
ax.set_xlabel(r'$\overline{CO2}$', labelpad=10);
ax.set_ylabel(r"Welfare", labelpad=10);
ax.legend(['$\overline{CO2}$', '$\overline{RES}$']);

# Plot 2: Zooming in
ax = plt.subplot(nrows, min(nplots,2), 2)
FC_RES_df.plot.area(ax = ax, legend=False, linewidth=0);
ax.set_xlim([0,baselineEmissions]);
ax.set_xlabel(r'$\overline{CO2}$', labelpad=10);
ax.set_ylabel(r"GJ fuel",labelpad=10);
handles, labels = ax.get_legend_handles_labels()
leg = ax.legend(handles[::-1], labels[::-1], loc=7, bbox_to_anchor = (1.4,0.5))
for line in leg.get_lines():
    line.set_linewidth(2)
fig.tight_layout()
if slides:
    fig.savefig(os.path.join(d['figs'], "L3_mBasicRES_slides.pdf"), facecolor='#FAFAFA',edgecolor='k')
else:
    fig.savefig(os.path.join(d['figs'], "L3_mBasicRES.pdf"),edgecolor='k')

## 3. mGFBasic

Load data:

In [26]:
kwargs = {'variables':  ['Fundamentals', 'Load','Generators_Other'],
          'variable2D': ['Generators_FuelMix'],
          'scalars': ['Scalars'],
          'maps': ['Generators_Categories']}
db = pyDbs.read.dbFromWB(os.path.join(d['data'], 'mGFBasic.xlsx'), kwargs)
pyDbs.readSets(db)

Run basic model:

In [27]:
m = mGFBasic.mSimple(db)
m() # go through standard steps

Solution status 0: Optimization terminated successfully. (HiGHS Status 7: Optimal)


Set up cost structre in the three components, add a column for total costs in order to solve by this - and then drop the column again:

In [28]:
costStructure = pd.DataFrame({'MC': m.db['mc'], 
                              'FOM': m.db['FOM'], 
                              'INV': pyDbs.adjMultiIndex.applyMult(m.db['InvestCost_A'],m.db['id2tech']).droplevel('tech')}).assign(
                Total = lambda x: x.sum(axis=1)).sort_values(by='Total')[['MC','FOM','INV']]

Plot stacked bar plot:

In [29]:
%%capture
one_graph()
if slides:
    fig, ax = plt.subplots(1,1,figsize = (14,8))
else:
    fig, ax = plt.subplots(1,1,figsize = (14,6))    
costStructure.plot.bar(stacked=True, ax=ax);
ax.set_xlabel('');
ax.set_ylabel('€/GJ');
fig.tight_layout();
if slides:
    fig.savefig(os.path.join(d['figs'],"L3_costStructure_slides.pdf"),facecolor='#FAFAFA',edgecolor='k')
else:
    fig.savefig(os.path.join(d['figs'],"L3_costStructure.pdf"),edgecolor='k')

Now, vary the allowed total capacity for wind and report the total system costs + investment in technologies:

In [30]:
loop = pd.Index(range(51), name = 'loop')
v0, vT = pd.Series([0], index = pd.Index(['Offshore wind'], name = 'tech')), pd.Series([db['Load'].sum()], index = pd.Index(['Offshore wind'], name = 'tech'))
capGrid = pyDbs.adjMultiIndex.addGrid(v0,vT,loop,'TechCap')
extract = ['GeneratingCapacity','Welfare']

We run the experiment using the ```loopSolvExtract``` method:

In [31]:
solution = m.loopSolveExtract(loop,[capGrid],extract)

Replace range loop with value of capacities:

In [32]:
[solution.__setitem__(k, pyDbs.adjMultiIndex.applyMult(v, pd.MultiIndex.from_frame(capGrid.droplevel('tech').reset_index())).droplevel('loop')) for k,v in solution.items()];

Plot

In [34]:
%%capture
mult_graphs()
if slides:
    fig, axes = plt.subplots(1, 2, figsize = (14, 8));
else:
    fig, axes = plt.subplots(1, 2, figsize = (14, 6));
# Plot 1:
ax = plt.subplot(1, 2, 1)
solution['Welfare'].plot(ax=ax, linewidth=3);
ax.set_ylabel("Welfare", labelpad=10);
ax.set_xlabel("Cap on wind",labelpad=10);

# Plot 2:
ax = plt.subplot(1, 2, 2)
solution['GeneratingCapacity'].unstack('id')[costStructure.index].plot.area(ax=ax, legend=False, linewidth=0);
ax.set_ylabel(r"$q_i$",labelpad=10);
ax.set_xlabel("Cap on wind",labelpad=10);
handles, labels = ax.get_legend_handles_labels()
leg = ax.legend(handles[::-1], labels[::-1], loc=7, bbox_to_anchor = (1.4,0.5))
for line in leg.get_lines():
    line.set_linewidth(2)
fig.tight_layout()
plt.subplots_adjust(right=1)
if slides:
    fig.savefig(os.path.join(d['figs'], "L3_capOnWind_slides.eps"), facecolor='#FAFAFA',edgecolor='k', bbox_inches='tight')
else:
    fig.savefig(os.path.join(d['figs'], "L3_capOnWind.pdf"),edgecolor='k', bbox_inches='tight')